# DART html crawler

> 기존 크롤링 코드를 html 크롤링으로 수정

1. directory 수정
    - 논의했던 내용대로 디렉토리 / 파일 구조 수정하기

2. html 파싱
    - class = tab-content 붙은것만 파싱하기 -> html 원본 그대로
    - tab-content > tab-content 있는 경우 하위 tab-content만 받아오는 방식으로

3. GPT 태우기
    - 기존에 썼던 프롬프트 사용
    - 화살표 태그 (arrow) 관련 → 로 표시해달라고 언급하기

In [1]:
## 의존성 라이브러리 설치 -- 버전에 맞는 Chromedriver 설치 필요
# !pip install selenium
# !pip install requests
# !pip install beautifulsoup4
# !pip install webdriver_manager

In [2]:
import requests               
from bs4 import BeautifulSoup as bs
import json
import os
from selenium.webdriver.common.by import By
from selenium import webdriver
import time

# For Chrome
#Selenium - Webdriver version 호환성
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [3]:
ROOT_PATH = '../DCM/dart/'
# menu_titles = [] 

def create_directory(path):
    if not os.path.exists(path):
        print(f'## Make directory in {path}')
        os.makedirs(path)

# 'dart/' 폴더 내에서 '{url_num}_'으로 시작하는 디렉토리 확인
def find_directory(directory, url_num):
    if not os.path.exists(directory):
        return False  # 디렉토리가 없으면 False 반환
    
    for name in os.listdir(directory):
        full_path = os.path.join(directory, name)
        if os.path.isdir(full_path) and name.startswith(url_num):
            return full_path  # 조건에 맞는 디렉토리가 있으면 full_path 반환
        
    return None  # 찾지 못하면 None 반환


def make_dir_path(url_num:str, title:str, tab_num=None, chapter_num=None, subtitle=None):
    print(f"## url_num {url_num}, title {title}, tab_num {tab_num}, chapter_num {chapter_num}, subtitle {subtitle}")

    # 상위 디렉토리 (e.g. 110_공시서류확인및서명)
    if tab_num == None and chapter_num == None:
        path = os.path.join(ROOT_PATH, url_num + '_' + title)
        create_directory(path)
    
    # 하위 디렉토리 (e.g. 110_1_제도안내)
    elif tab_num != None and chapter_num == None:
        # url_num이 같은 상위 디렉토리 서칭하기
        parent_dir = find_directory(ROOT_PATH, url_num)
        if parent_dir != None:
            path = os.path.join(parent_dir, url_num + '_' + tab_num + '_' + title)
            create_directory(path)
        else:
            raise ValueError("No parent directory exists!")
    
    # 탭 안에 하위 탭 (이하 챕터) 있는 경우 (e.g. 220_1-1_제도안내_개요)
    # # ../DCM/dart/220_2-1_세부업무_Ⅰ 개요 
    else:
        # url_num이 같은 상위 디렉토리 서칭하기
        parent_dir = find_directory(ROOT_PATH, url_num)
        if parent_dir != None:
            path = os.path.join(parent_dir, url_num + '_' + tab_num + '-' + chapter_num  + '_' + title + '_' + subtitle)
            create_directory(path)
        else:
            raise ValueError("No parent directory exists!")

    return path

In [4]:
import re

def get_page_html(url):
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = bs(html, 'html.parser')
        return soup.prettify()

    else: 
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

def get_url_num(url):
    return re.findall(r'\d+', url)[0]

- 셀레니움으로 클릭은 만들어야 할 듯
- 이후 url 뒷 번호 따오기
- 페이지 제목 : 공시서류확인및서명 - #contents > div.sub-top > div > h2
- 탭 제목 : 제도안내 - #contents > div.content > div.page-tab > ul > li:nth-child(1) > a
- 챕터 제목 : I 개요 - #content-tab-1 > div.sub-top-tab > ul > li:nth-child(1) > a

- html 크롤링 이전에 앞에 "제목 : 홈 > 공시 실무자 > 주요사항보고서" 넣기

``` python
# menu 제목 추출
    menu_title = driver.find_element(By.CLASS_NAME, "page-navigation")
    menu_title = menu_title.text.strip().replace(' ', '')
    menu_titles.append(menu_title)  # 제목을 리스트에 추가
    print(f"제목: {menu_title}")

```

In [5]:
def html_parsing(html, url_num, parent_title, parent_path):
    soup = bs(html, 'html.parser')

    li_elements = soup.select("#contents > div.content > div.page-tab > ul > li")

    # 탭 없는 경우 존재 (menu=500)
    if len(li_elements) == 0:
        print(f'## Tabs not exists!')
        
        content_element = soup.select("#contents > div.content > div")[0].prettify()

        with open(os.path.join(parent_path, parent_title+".txt"), "w") as file:
            print(f"## {parent_path} 에 {os.path.join(parent_path, parent_title+'.txt')} 저장 ##")
            file.write(content_element)

    else:
        # 탭 단위
        for idx, li in enumerate(li_elements, start=1):
            tab_title = li.get_text(strip=True).strip().replace(' ', '')
            print(f'## 현재 크롤링 중인 탭 : {tab_title}')
            path = make_dir_path(url_num, tab_title, str(idx))
            content = soup.select_one(f"#content-tab-{idx}").prettify()
            
            sub_soup = bs(content, 'html.parser')

            sub_li_elements = sub_soup.select(f"#content-tab-{idx} > div.sub-top-tab > ul > li")

            if sub_li_elements:
                print("## 하위 탭 존재 ##")
                for sub_idx, sub_li in enumerate(sub_li_elements, start=1):
                    subtab_title = sub_li.get_text(strip=True).strip().replace(' ', '')
                    path = make_dir_path(url_num, tab_title, str(idx), str(sub_idx), subtab_title)
                    sub_content = soup.select_one(f"#content-tab-{idx}-{sub_idx}").prettify()

                    with open(os.path.join(path, tab_title + "_" + subtab_title +".txt"), "w") as file:
                        print(f"## {path} 에 {os.path.join(path, tab_title + '_' + subtab_title +'.txt')} 저장 ##")
                        file.write(sub_content)
                    
            else:
                with open(os.path.join(path, tab_title+".txt"), "w") as file:
                    print(f"## {path} 에 {os.path.join(path, tab_title+'.txt')} 저장 ##")
                    file.write(content)
    

In [6]:
import requests
import re

options = Options()
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# URL 설정
URL = "https://dart.fss.or.kr/info/selectGuide.do"


# 페이지 열기
driver.get(URL)
time.sleep(2)


# 메뉴 열기 버튼 클릭
menu = driver.find_element(By.CLASS_NAME, 'btn-menu')
menu.click()
time.sleep(1)


# ul li 안의 ul li 안의 a 태그를 가져오기
li_elements = driver.find_elements(By.CSS_SELECTOR, 'ul li ul li')

for idx, li in enumerate(li_elements):
    a = li.find_element(By.TAG_NAME, 'a')
    link_text = a.get_attribute('innerText').strip()
    link_href = a.get_attribute('href')
    
    if link_text:
        print(f"텍스트: {link_text}, 링크: {link_href}")

        # url 넣고 bs4로 파싱
        # 탭 전체 기준 디렉토리 생성
        try:
            url_num = re.findall(r'\d+', link_href)[0]
            
        except:
            print(f"공시업무 게시판 제외")
            continue

        path = make_dir_path(url_num, link_text.replace(' ', '')) # 110_공시서류확인및서명
        # 전처리는 bs4단에서 하기
        html = get_page_html(link_href)
        html_parsing(html, url_num, link_text.replace(' ', ''), path)
        
    else:
        print(f"빈 텍스트: 링크: {link_href}")

print("## 크롤링 완료! ##")

driver.quit()

텍스트: 공시서류 확인 및 서명, 링크: https://dart.fss.or.kr/info/main.do?menu=110
## url_num 110, title 공시서류확인및서명, tab_num None, chapter_num None, subtitle None
## Make directory in ../DCM/dart/110_공시서류확인및서명
## 현재 크롤링 중인 탭 : 제도안내
## url_num 110, title 제도안내, tab_num 1, chapter_num None, subtitle None
## Make directory in ../DCM/dart/110_공시서류확인및서명/110_1_제도안내
## ../DCM/dart/110_공시서류확인및서명/110_1_제도안내 에 ../DCM/dart/110_공시서류확인및서명/110_1_제도안내/제도안내.txt 저장 ##
## 현재 크롤링 중인 탭 : 업무절차
## url_num 110, title 업무절차, tab_num 2, chapter_num None, subtitle None
## Make directory in ../DCM/dart/110_공시서류확인및서명/110_2_업무절차
## ../DCM/dart/110_공시서류확인및서명/110_2_업무절차 에 ../DCM/dart/110_공시서류확인및서명/110_2_업무절차/업무절차.txt 저장 ##
## 현재 크롤링 중인 탭 : 작성사례
## url_num 110, title 작성사례, tab_num 3, chapter_num None, subtitle None
## Make directory in ../DCM/dart/110_공시서류확인및서명/110_3_작성사례
## ../DCM/dart/110_공시서류확인및서명/110_3_작성사례 에 ../DCM/dart/110_공시서류확인및서명/110_3_작성사례/작성사례.txt 저장 ##
텍스트: 이사의 경영진단 및 분석의견, 링크: https://dart.fss.or.kr/info/main.do?menu=120


def make_dir_path(url_num:str, title:str, tab_num=None, chapter_num=None, subtitle=None):

In [ ]:
html = get_page_html("https://dart.fss.or.kr/info/main.do?menu=410")
html_parsing(html, "410")